In [10]:
import torch
import torchvision.models as models
import torch.nn as nn

model = models.resnet50(pretrained=True)
model.fc = nn.Identity()

from torchvision import transforms

preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

from PIL import Image
img_database = ["/content/chong_1.pic.jpg", "/content/chong_2.pic.jpg","/content/chong_3.pic.jpg",
                "/content/erdai_1.jpg", "/content/erdai_2.jpg","/content/erdai_3.jpg",
                "/content/meiqiu1.pic.jpg","/content/meiqiu2.pic.jpg","/content/meiqiu3.pic.jpg",
                "/content/tusi1.pic.jpg", "/content/tusi2.pic.jpg", "/content/tusi3.pic.jpg",
                "/content/datou1.pic.jpg","/content/datou2.pic.jpg","/content/datou3.pic.jpg"]

cat_ids = [0,0,0,1,1,1,2,2,2,3,3,3,4,4,4]
total_img = []
for img in img_database:
    image = Image.open(img)
    x = preprocess(image)
    x = x.unsqueeze(0)
    total_img.append(x)
sum_img = torch.cat(total_img,dim = 0)
tensor_database = model(sum_img)

cat_data = {
    'cat_ids': cat_ids,
    'cat_emb': tensor_database
}

torch.save(cat_data, "/content/cats_data.pt")

image_testing = "/content/2151689219780_.pic.jpg"
image = Image.open(image_testing)
x = preprocess(image)
x = x.unsqueeze(0)
emb_testing = x
with torch.no_grad():
    emb_testing = model(x)


loaded_tensor_database = torch.load("/content/cats_data.pt")
cosine_similarities = torch.nn.functional.cosine_similarity(emb_testing, loaded_tensor_database['cat_emb'])
print(cosine_similarities)
anslist = cosine_similarities.tolist()
ans = max(anslist)
ans_i = anslist.index(ans)
print (ans,ans_i)
print (f"The rank 1 cat has catID {loaded_tensor_database['cat_ids'][ans_i]} and similarity {ans}")

k = 5
res = torch.topk(cosine_similarities,k)

print(res)
IDlist = res.indices.tolist()
val_list = res.values.tolist()
count = [0]*5

for i in range(k):
  ID = IDlist[i]
  val = val_list[i]
  count[loaded_tensor_database['cat_ids'][ID]]+=1
  print (f"The rank {i+1} cat has catID {loaded_tensor_database['cat_ids'][ID]} and similarity {val}")

print(count)
resknn = count.index(max(count))
print(f"the final predicted cat is of catsID{resknn}")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


tensor([0.8539, 0.7819, 0.8359, 0.8189, 0.8473, 0.8348, 0.8461, 0.8463, 0.8389,
        0.8179, 0.8445, 0.8216, 0.8059, 0.8580, 0.7835],
       grad_fn=<SumBackward1>)
0.8580335378646851 13
The rank 1 cat has catID 4 and similarity 0.8580335378646851
torch.return_types.topk(
values=tensor([0.8580, 0.8539, 0.8473, 0.8463, 0.8461], grad_fn=<TopkBackward0>),
indices=tensor([13,  0,  4,  7,  6]))
The rank 1 cat has catID 4 and similarity 0.8580335378646851
The rank 2 cat has catID 0 and similarity 0.8538737893104553
The rank 3 cat has catID 1 and similarity 0.8473202586174011
The rank 4 cat has catID 2 and similarity 0.8462850451469421
The rank 5 cat has catID 2 and similarity 0.8461136817932129
[1, 1, 2, 0, 1]
the final predicted cat is of catsID2
